In [ ]:
import pyreadr
import numpy as np

result = pyreadr.read_r("SMRi ordered - Brazil")   # também lê .RData
obj = next(iter(result.values()))        # pega o único objeto do .rds

# se for um data frame R, vira um pandas.DataFrame
import pandas as pd
df = obj if isinstance(obj, pd.DataFrame) else None
df

In [ ]:
vars_cols = df.var()   
vars_cols.head()

In [ ]:
df_std = df.copy()

#vamos padronizar as colunas 
mu = df_std.mean(axis=0)
sd = df_std.std(axis=0, ddof=0)  # desvio padrão populacional (ddof=0)

# evita divisão por zero (coluna constante). Aqui descartamos constantes.
keep = sd > 0

# Y é o DataFrame padronizado
Y = ((df_std.loc[:, keep] - mu[keep]) / sd[keep]).fillna(0.0)
print(Y.values, Y.shape)
Y.var().head()

In [ ]:
# Agora sim, podemos fazer a SVD
from numpy.linalg import svd

# X = M S Vt
M, S, Vt = svd(Y.values, full_matrices=False)
print(M.shape, S.shape, Vt.shape)  # M:(n×q), S:(q,), Vt:(q×q)

In [ ]:
# --- 3) Gavish–Donoho para escolher K ---
n, q = Y.shape

beta = min(q / n, n / q)
sigma_median = np.median(S)
omega = 0.56*beta**3 - 0.95*beta**2 + 1.82*beta + 1.43
tau = omega * sigma_median
K = np.sum(S >= tau)
print(f"n={n}, q={q}, beta={beta:.3f}, omega={omega:.3f}, tau={tau:.3f}, K={K}")